In [1]:
import pandas as pd
import numpy as np

In [8]:
emoji_dataset = pd.read_csv("go_emotions_dataset.csv")

In [33]:
bad_indices = emoji_dataset.query('example_very_unclear == False').index
emoji_updated1 = emoji_dataset.iloc[bad_indices,:].drop(columns = ['example_very_unclear', 'id'])
dict = {'Sentences' : emoji_updated1.iloc[:,0].to_numpy(), 'Emotions': emoji_updated1.iloc[:,1:].idxmax(1).to_numpy()}
emoji_updated2 = pd.DataFrame(dict)
emoji_updated2

,Sentences,Emotions
0,That game hurt.,sadness
1,"You do right, if you don't care then fuck 'em!",neutral
2,Man I love reddit.,love
3,"[NAME] was nowhere near them, he was by the Fa...",neutral
4,Right? Considering it’s such an important docu...,gratitude
...,...,...
207809,"Well, I'm glad you're out of all that now. How...",joy
207810,Everyone likes [NAME].,love
207811,Well when you’ve imported about a gazillion of...,caring
207812,That looks amazing,admiration


In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
sentences = np.array(emoji_updated2['Sentences']).astype(str)
category = np.array(emoji_updated2['Emotions'].astype('category').cat.codes).astype('int8')
vect = CountVectorizer().fit(sentences)
X_train_vect = vect.transform(sentences)
tf = TfidfTransformer().fit(X_train_vect)
X_train = tf.transform(X_train_vect).astype(np.float32).todense()
#full_transformed = np.append(X_train, category.reshape(207814,1), axis=1)

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
X_train, X_test, y_train, y_test = train_test_split(X_train, category, test_size=0.1)
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [92]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(256, input_shape=(30698,), activation="relu"))
model.add(Dense(28, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer='adam',
              metrics=["accuracy"])

In [93]:
H = model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=100, batch_size=32, verbose = 1)

Epoch 1/100
5845/5845 [==============================] - 136s 23ms/step - loss: 2.1635 - accuracy: 0.3873 - val_loss: 2.0221 - val_accuracy: 0.4133
Epoch 2/100
5845/5845 [==============================] - 139s 24ms/step - loss: 1.7644 - accuracy: 0.4531 - val_loss: 2.0496 - val_accuracy: 0.4105
Epoch 3/100
5845/5845 [==============================] - 138s 24ms/step - loss: 1.5967 - accuracy: 0.4765 - val_loss: 2.1153 - val_accuracy: 0.4048
Epoch 4/100
5845/5845 [==============================] - 140s 24ms/step - loss: 1.4910 - accuracy: 0.4895 - val_loss: 2.1965 - val_accuracy: 0.3996
Epoch 5/100
5845/5845 [==============================] - 140s 24ms/step - loss: 1.4177 - accuracy: 0.4978 - val_loss: 2.2606 - val_accuracy: 0.3944
Epoch 6/100
5845/5845 [==============================] - 140s 24ms/step - loss: 1.3635 - accuracy: 0.5043 - val_loss: 2.3144 - val_accuracy: 0.3957
Epoch 7/100
5845/5845 [==============================] - 138s 24ms/step - loss: 1.3211 - accuracy: 0.5079 - val_

KeyboardInterrupt: 

(187032, 30698)